# Demo of function *indices_ts_stats* from *./swiss_utils/data_cube_utilities/sdc_utilities.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy.__

This function compute statistics through time (for given period of time) for indices of given products.

It requires a dataset to be analyzed `ds`, a dictionary with seasons names as keys and lists of months as values `sm_dict`, a dictionnary with the indices names as keys and related functions as values `idces_dict`, a list of statistical functions to be applied to the data `stats`, optionaly a list of percentiles to calculate `nanperc`. And will return a dataset with the requested statistics per season and indice.

This notebook will also demonstrate how annual time series car be generated for a list of indices, seasons and statistics.

*****

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `indices_ts_stats?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_advutils.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

import os
import shutil

import numpy as np

from datetime import datetime

from swiss_utils.data_cube_utilities.sdc_utilities import load_multi_clean, write_geotiff_from_xr

import datacube
dc = datacube.Datacube()

# silence warning (not recommended during development)
import warnings
warnings.filterwarnings("ignore")

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_utilities import indices_ts_stats

The next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

You can generate it in three ways:
1. manually from scratch,
2. by manually copy/pasting the final cell content of the [config_tool](config_tool.ipynb) notebook,
3. by loading the final cell content of the [config_tool](config_tool.ipynb) notebook using the magic `# %load config_cell.txt`.

In [ ]:
%load config_cell.txt

In [ ]:
# indices_ts_stats arguments

# Indices - formula dictionnary
# measurements in previous cell must include all bands in the dictionnary
idces_dict = {'NDVI': '(ds.nir - ds.red) / (ds.nir + ds.red)',
              'NDWI': '(ds.green - ds.nir) / (ds.green + ds.nir)',
              'NDBI': '(ds.swir2 - ds.nir) / (ds.swir2 + ds.nir)'}

# Seasons - months dictionnary
sm_dict = {'annual': ('all'),
           'winter': (1, 2, 12),
           'spring': (3, 4, 5),
           'summer': (6, 7, 8),
           'autumn': (9, 10, 11)}

# Stats list (without '' for numpy functions, 'range' was manually implemented)
# and requires nanpercs list
# stats = [np.nanmin, np.nanmax, np.nanmedian, np.nanmean, np.nanstd, 'range']
# nanpercs = [5, 25, 75, 95]
stats = [np.nanmin, np.nanmax, np.nanpercentile, 'range']
nanpercs = [10]

In [ ]:
# Load the dataset and clean it

ds_in, clean_mask = load_multi_clean(dc = dc, products = product,
                                     time = [start_date, end_date],
                                     lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                     measurements = measurements)
del clean_mask
ds_in = ds_in.where(ds_in >= 0) # keep only positive values
ds_in = ds_in.dropna('time', how='all') # drop scenes without data

In [ ]:
# Compute statistics

ds_stats = indices_ts_stats(ds_in, sm_dict, idces_dict, stats, nanpercs)
ds_stats

**Then it is easy to generate annual statistics for a list of indices using `indices_ts_stats` function.**

Here is a mini-reminder of Landsat and Sentinel 2 time range (for Swiss Data Cube):

| product          | complete years |
|------------------|----------------|
| ls5_ledaps_swiss | 1985-2010      |
| ls7_ledaps_swiss | 2000-2020      |
| ls8_lasrc_swiss  | 2016-2021      |
| s2_l2a_10m_swiss | 2016-2021      |

In [ ]:
# update config cell

out_dir = 'indices_ts_stats'

products = ['ls8_lasrc_swiss', 'ls7_ledaps_swiss']

years = [2015, 2017] # [min, max] (range)

In [ ]:
# Remove previous output if any

if os.path.isdir(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir)

if os.path.isfile(f'{out_dir}.zip'):
    os.remove(f'{out_dir}.zip')

In [ ]:
# Process year per year

for y in range(years[0], years[1] + 1):
    print(y)
    ds_year, clean_mask = load_multi_clean(dc = dc, products = products,
                                           time = [datetime.strptime(f'{y}-1-1', '%Y-%m-%d'),
                                                   datetime.strptime(f'{y + 1}-1-1', '%Y-%m-%d')],
                                           lon = [min_lon, max_lon], lat = [min_lat, max_lat],
                                           measurements = measurements)
    del clean_mask
    ds_year = ds_year.where(ds_year >= 0) # keep only positive values
    ds_year = ds_year.dropna('time', how='all') # drop scenes without data
    
    # compute statistics
    ds_stats = indices_ts_stats(ds_year, sm_dict, idces_dict, stats, nanpercs)
    
    # Export as geotiff
    write_geotiff_from_xr(os.path.join(out_dir, f"ds_stats_{y}.tif"),
                          ds_stats,
                          crs = ds_year.crs,
                          compr = 'DEFLATE')

In [ ]:
# Compress <out_dir> in a single zip file for easier download

shutil.make_archive(out_dir, 'zip', out_dir)